## Finding the start of a flash drought using ET Criteria

In [1]:
#importing packages
import numpy as np
import xarray as xr
import matplotlib.pyplot as plt
import pandas as pd
import dask.distributed as dsk

# Start Client/ dask runs together with xarray to improve performance. Only need to initialize once here.
clnt = dsk.Client()

/g/data/hh5/public/apps/miniconda3/envs/analysis3-22.10/lib/python3.9/site-packages/distributed/node.py:182: UserWarning: Port 8787 is already in use.
Perhaps you already have a cluster running?
Hosting the HTTP server on port 34781 instead
  warnings.warn(


In [17]:
# path to directory of AWRA model data
awra_dir = '/g/data/fj8/BoM/AWRA/DATA/'


# selecting a time slice and specific latitude and longitude to work on a single grid cell
time_slice = slice('1980-01-01', '2022-12-31')
lat_sel = -37
lon_sel = 148

In [18]:
#load in filtered FD start dates from quantiles method
fd = xr.open_dataset('/g/data/w97/aj8747/fd_summer_project/fd_sm_start_dates.nc')

#check how many FD values you have. ie. how many potential start dates 
fd['sm_pct'].dropna(dim = 'time') 

<xarray.DataArray 'sm_pct' (time: 151)>
array([0.442254, 0.430588, 0.437388, 0.430019, 0.422903, 0.414275, 0.447263,
       0.440526, 0.866418, 0.852368, 0.838957, 0.694434, 0.685912, 0.677313,
       0.694393, 0.703106, 0.703042, 0.705589, 0.773731, 0.49892 , 0.487598,
       0.476865, 0.465914, 0.456605, 0.447173, 0.761945, 0.797246, 0.786884,
       0.774444, 0.771539, 0.768736, 0.757441, 0.746563, 0.447737, 0.439907,
       0.794647, 0.782575, 0.770801, 0.76494 , 0.622896, 0.832396, 0.818147,
       0.805317, 0.804382, 0.791185, 0.777686, 0.764103, 0.750332, 0.736929,
       0.722805, 0.711222, 0.697555, 0.453799, 0.444104, 0.436767, 0.444863,
       0.438863, 0.69726 , 0.790419, 0.798684, 0.791818, 0.75021 , 0.453509,
       0.445574, 0.439151, 0.742262, 0.735761, 0.727865, 0.718583, 0.709044,
       0.716273, 0.703962, 0.636984, 0.411953, 0.414448, 0.41423 , 0.405358,
       0.757959, 0.74685 , 0.756818, 0.755451, 0.698313, 0.42569 , 0.4171  ,
       0.409157, 0.401737, 0.499922, 0.491523, 0.483101, 0.47526 , 0.466997,
       0.795096, 0.783971, 0.773054, 0.779781, 0.75792 , 0.745792, 0.732432,
       0.753963, 0.74118 , 0.727909, 0.767075, 0.75272 , 0.73856 , 0.724963,
       0.711828, 0.747821, 0.742647, 0.75344 , 0.755938, 0.742326, 0.728272,
       0.715965, 0.702373, 0.688701, 0.485645, 0.486888, 0.485915, 0.479473,
       0.711646, 0.704781, 0.844177, 0.820843, 0.807455, 0.794851, 0.743259,
       0.730647, 0.542118, 0.458793, 0.445845, 0.492533, 0.796373, 0.789364,
       0.778287, 0.767366, 0.717183, 0.698467, 0.689417, 0.680444, 0.671727,
       0.754482, 0.755945, 0.744434, 0.78313 , 0.770033, 0.760005, 0.750204,
       0.810563, 0.799359, 0.786625, 0.761803], dtype=float32)
Coordinates:
  * time       (time) datetime64[ns] 1980-02-15 1980-02-16 ... 2020-09-03
    latitude   float64 ...
    longitude  float64 ...
Attributes:
    name:           sm_pct
    units:          Fraction of fullness
    long_name:      Total rootzone soil moisture (0-100 cm)
    standard_name:  sm_pct

In [19]:
# Convert to pandas dataframe 
df = fd.to_dataframe()
#df.drop(columns = ['latitude','longitude'], inplace=True)
df['num_index'] = range(0,len(df)) #create a new column which indicates the length of the columns

In [20]:
#load in evapotranspiratin data 
etot_dir = awra_dir + 'SCHEDULED-V6/etot_*.nc'
ds_et = xr.open_mfdataset(etot_dir, chunks = {'time':'200mb'}, parallel = True)

# data at a grid point and a specific time
da_et = ds_et.sel(latitude = lat_sel, longitude = lon_sel, time = time_slice)
da_et = da_et.load()

#array of ET
ET = da_et['etot']
ET

<xarray.DataArray 'etot' (time: 15706)>
array([2.30656389, 2.60561533, 3.51802028, ..., 2.69179201, 2.63133854,
       5.98783821])
Coordinates:
  * time       (time) datetime64[ns] 1980-01-01 1980-01-02 ... 2022-12-31
    latitude   float64 -37.0
    longitude  float64 148.0
Attributes:
    long_name:      etot
    units:          mm
    name:           etot
    standard_name:  etot

In [21]:
# Putting the ET values in the df dataframe

df['ET'] = np.nan #adding new column called ET 
for row in df.index: #index is datetime for the dataframe, so for row in each datetime
    df.loc[row, 'ET'] = ET.sel(time=row)
    
df

,latitude,longitude,sm_pct,num_index,ET
time,,,,,
1980-01-01,-37.0,148.0,NaN,0,2.306564
1980-01-02,-37.0,148.0,NaN,1,2.605615
1980-01-03,-37.0,148.0,NaN,2,3.518020
1980-01-04,-37.0,148.0,NaN,3,5.368407
1980-01-05,-37.0,148.0,NaN,4,3.738397
...,...,...,...,...,...
2022-12-27,-37.0,148.0,NaN,15701,5.087646
2022-12-28,-37.0,148.0,NaN,15702,3.178666
2022-12-29,-37.0,148.0,NaN,15703,2.691792


In [22]:
#creating monthly and dayofyear climatology

clim_doy = df['ET'].groupby(df.index.dayofyear).mean() 
clim_mon = df['ET'].groupby(df.index.month).mean() 

In [66]:
#Creating a column for when ET matches the criteria
df['ET_above80pct'] = np.nan
for row in df.index:
    clim_value = clim_doy[row.dayofyear]
    if ~np.isnan(df.loc[row,'sm_pct']): 
        if df.loc[row,'ET'] >= clim_value*0.8: #if above 0.8*clim
            if row -  pd.to_timedelta(20, unit='d') > df.index[0]: 
                df.loc[row-pd.to_timedelta(20, unit='d'):row, 'ET_above80pct'] = df.loc[row-pd.to_timedelta(20, unit='d'):row]['ET'] #then it is equal to 1
            
df.dropna()[0:60]

#need if the length of a period is 20 days keep it, if it is less than 20 days, do not keep it

,latitude,longitude,sm_pct,num_index,ET,ET_above80pct,ET_binary
time,,,,,,,
1980-02-15,-37.0,148.0,0.442254,45,3.664665,3.664665,0
1980-02-16,-37.0,148.0,0.430588,46,3.485145,3.485145,0
1980-03-14,-37.0,148.0,0.437388,73,1.840308,1.840308,0
1980-03-15,-37.0,148.0,0.430019,74,1.606545,1.606545,0
1980-03-16,-37.0,148.0,0.422903,75,1.535982,1.535982,0
1980-03-17,-37.0,148.0,0.414275,76,2.261897,2.261897,0
1981-04-11,-37.0,148.0,0.447263,466,1.546262,1.546262,0
1981-04-12,-37.0,148.0,0.440526,467,1.982643,1.982643,0
1981-09-01,-37.0,148.0,0.866418,609,1.464121,1.464121,0


SyntaxError: 'return' outside function (3250908839.py, line 4)